# Validation of Generated_Data

### Targets and Values

In [1]:
import os

In [2]:
num_samples_to_generate=200
# Target = "Normal"
output_dirs = [(r'Generated\patient.npy', r'Real\patient_cls.npy',)]
base_path = r'..\Dataset'


### Utility Function

In [3]:
signal_length = 500

In [4]:
for output_dir in output_dirs:
    print(output_dir)

('Real\\normal_cls.npy', 'Real\\patient_cls.npy')


In [5]:
import numpy as np
from tqdm import tqdm
import os

def calc_avg(func):
    global base_path
    global signal_length
    for output_dir in output_dirs:
        
        data1 = np.load(os.path.join(base_path, output_dir[1]))
        data2 = np.load(os.path.join(base_path, output_dir[0]))


        real_to_gen = []
        gen_to_rand = []
        real_to_rand = []
        max_rep = 500
        current = 0
        for eeg_signal1 in tqdm(data1):
            current+=1
            if current > max_rep:
                break
            for eeg_signal2 in data2:
                eeg_rand = 2 * np.random.rand(19, signal_length) - 1
                real_to_gen.append(func(eeg_signal2,eeg_signal1))
                gen_to_rand.append(func(eeg_rand,eeg_signal1))
                real_to_rand.append(func(eeg_signal2,eeg_rand))

        print(f"{'='*20}{output_dir}{'='*20}")
        real_to_gen = sum(real_to_gen)/len(real_to_gen)
        gen_to_rand = sum(gen_to_rand)/len(gen_to_rand)
        real_to_rand = sum(real_to_rand)/len(real_to_rand)
        print(f"\t{f'{output_dir[1]} to {output_dir[0]}:':<50} \t {real_to_gen: >25.15f}")
        print(f"\t{f'{output_dir[0]} to Random:':<50} \t {gen_to_rand: >25.15f}")
        print(f"\t{f'{output_dir[1]}' ' to Random:':<50} \t {real_to_rand: >25.15f}")

### Euclidian Distance

In [6]:
import numpy as np
from random import randint 
import os

def euc_dis(signal1, signal2):
    signal1_flat = signal1.flatten()
    signal2_flat = signal2.flatten()
    return np.linalg.norm(signal1_flat - signal2_flat)

calc_avg(euc_dis)

100%|██████████| 100/100 [00:00<00:00, 182.66it/s]

====================('Real\\normal_cls.npy', 'Real\\patient_cls.npy')====================
	Real\patient_cls.npy to Real\normal_cls.npy:       	        18.607836638119878
	Real\normal_cls.npy to Random:                     	        58.018205966878512
	Real\patient_cls.npy to Random:                    	        57.546436395227715


### Cosine Similarity

In [7]:
from scipy.spatial.distance import cosine
import numpy as np

def cosine_sim(signal1, signal2):
    signal1_flat = signal1.flatten()
    signal2_flat = signal2.flatten()

    # Calculate cosine distance
    cosine_distance = cosine(signal1_flat, signal2_flat)

    # Convert cosine distance to cosine similarity
    return 1 - cosine_distance
calc_avg(cosine_sim)


100%|██████████| 100/100 [00:00<00:00, 160.27it/s]

====================('Real\\normal_cls.npy', 'Real\\patient_cls.npy')====================
	Real\patient_cls.npy to Real\normal_cls.npy:       	        -0.043784943303589
	Real\normal_cls.npy to Random:                     	        -0.000081443452530
	Real\patient_cls.npy to Random:                    	         0.000093124807360


### Pearson Correlation Coefficient

In [8]:
from scipy.stats import pearsonr
import numpy as np

def pearson_coef(signal1, signal2):
    signal1_flat = signal1.flatten()
    signal2_flat = signal2.flatten()

    # Calculate Pearson correlation coefficient using scipy
    pearson_corr, p_value = pearsonr(signal1_flat, signal2_flat)
    return pearson_corr

calc_avg(pearson_coef)




100%|██████████| 100/100 [00:11<00:00,  8.84it/s]

====================('Real\\normal_cls.npy', 'Real\\patient_cls.npy')====================
	Real\patient_cls.npy to Real\normal_cls.npy:       	        -0.000490276894657
	Real\normal_cls.npy to Random:                     	         0.000080723999202
	Real\patient_cls.npy to Random:                    	         0.000052029952183
